In [1]:
import os

# 1. Bring YOLOv5 onto the GPU machine
print("Cloning YOLOv5...")
!git clone https://github.com/ultralytics/yolov5
%cd /content/yolov5
!pip install -r requirements.txt

Cloning YOLOv5...
Cloning into 'yolov5'...
remote: Enumerating objects: 17817, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17817 (delta 12), reused 6 (delta 6), pack-reused 17796 (from 3)
Receiving objects: 100% (17817/17817), 17.01 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (12143/12143), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


In [2]:
# 2. Install Roboflow and download the Road Damage dataset
print("Downloading Road Hazard dataset...")
!pip install roboflow
from roboflow import Roboflow

# ⚠️ PASTE YOUR API KEY HERE:
rf = Roboflow(api_key="1I6CRwFB54fg4xm9DgcI")

# Pulling a highly-rated Pothole & Road Crack dataset pre-formatted for YOLOv5
project = rf.workspace("pothole-detection-bygur").project("pothole-detection-yolo-v5")
version = project.version(1)
dataset = version.download("yolov5")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 91.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.92
    Uninstalling opencv-python-headless-4.13.0.92:
      Successfully uninstalled opencv-python-headless-4.13.0.92
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...
Generating version still in progress. Progress: 100.0%

KeyboardInterrupt: 

In [2]:
# 3. Disable the Weights & Biases login prompt
os.environ["WANDB_DISABLED"] = "true"

In [3]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("lorenzoarcioni/road-damage-dataset-potholes-cracks-and-manholes")

print("Path to dataset files:", path)




Using Colab cache for faster access to the 'road-damage-dataset-potholes-cracks-and-manholes' dataset.
Path to dataset files: /kaggle/input/road-damage-dataset-potholes-cracks-and-manholes


In [4]:

import os
import kagglehub

# 2. Download the Road Hazard dataset using kagglehub
print("Downloading Road Hazard dataset from Kaggle...")
path = kagglehub.dataset_download("lorenzoarcioni/road-damage-dataset-potholes-cracks-and-manholes/versions/4")
print(f"Dataset downloaded to: {path}")

# Create a symbolic link for labels to match YOLOv5's expected structure
!ln -s {path}/data/labels-YOLO {path}/data/labels

# 3. Disable the Weights & Biases login prompt
os.environ["WANDB_DISABLED"] = "true"

# 4. Map the path to the YOLO dataset folder
# Update train and val paths to point to the correct image directory
yaml_content = f"""
train: {path}/data/images
val: {path}/data/images

nc: 3
names: ['Pothole', 'Crack', 'Manhole']
"""

yaml_path = "/content/yolov5/road_data.yaml"

# Ensure the directory exists before attempting to write the file
os.makedirs(os.path.dirname(yaml_path), exist_ok=True)

with open(yaml_path, 'w') as f:
   f.write(yaml_content)

print(f"✅ Created custom config: {yaml_path}")

# 5. Start the GPU Training!
print("🚀 Starting YOLOv5 Training Phase on GPU...")
!python train.py --img 640 --batch 16 --epochs 50 --data {yaml_path} --weights yolov5s.pt --name smart_car_vision




Streaming output truncated to the last 5000 lines.
      30/49      3.52G    0.04761    0.02082   0.007116         89        640:  60% 76/126 [00:27<00:16,  3.08it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      30/49      3.52G    0.04762    0.02085   0.007096         70        640:  61% 77/126 [00:28<00:18,  2.62it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      30/49      3.52G    0.04771    0.02087   0.007084         77        640:  62% 78/126 [00:28<00:16,  2.84it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      30/49      3.52G    0.04773 